# 1. Load packages

In [1]:
from analysis_tools.common import *
import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa
import sklearn
import cv2
import os

%load_ext autoreload
%autoreload 2

np.random.seed(RANDOM_STATE)
tf.random.set_seed(RANDOM_STATE)
sklearn.random.seed(RANDOM_STATE)

for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)
    
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2')


# 2. Load dataset

In [2]:
train_full_data_meta = pd.read_csv(join(PATH.input, 'train_df.csv'), index_col=0)
test_data_meta       = pd.read_csv(join(PATH.input, 'test_df.csv'), index_col=0)

with ProgressBar():
    X_train_full = compute(*[delayed(cv2.imread)(path) for path in ls_file(PATH.train)])
    X_test       = compute(*[delayed(cv2.imread)(path) for path in ls_file(PATH.test)])
y_train_full = train_full_data_meta[['label']]
    
print("- Number of train full data:", len(X_train_full))
print("- Number of test data:", len(X_test))

[########################################] | 100% Completed |  7.7s
[########################################] | 100% Completed |  3.6s
- Number of train full data: 4277
- Number of test data: 2154


# 3. Training

In [3]:
from sklearn.preprocessing import OneHotEncoder

IMG_SIZE    = 512
input_shape = (IMG_SIZE, IMG_SIZE, 3)

with ProgressBar():
    X_train_full = np.array(compute(*[delayed(cv2.resize)(X, [IMG_SIZE, IMG_SIZE]) for X in X_train_full]))
    X_test       = np.array(compute(*[delayed(cv2.resize)(X, [IMG_SIZE, IMG_SIZE]) for X in X_test]))
y_enc = OneHotEncoder(sparse=False, dtype=bool)
y_train_full = y_enc.fit_transform(y_train_full)
n_classes = len(y_enc.categories_[0])
print("- Number of classes:", n_classes)

[########################################] | 100% Completed |  2.3s
[########################################] | 100% Completed |  0.9s
- Number of classes: 88


In [4]:
aug_model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.RandomFlip('horizontal_and_vertical'),
    keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

def preprocess(ds, training, batch_size, augment=True):
    ds = ds.cache().batch(batch_size)
    if training:
        ds = ds.shuffle(buffer_size=1000).prefetch(tf.data.AUTOTUNE)
        if augment:
            ds = ds.map(lambda X, y, sw: (aug_model(X), y, sw), num_parallel_calls=tf.data.AUTOTUNE)
    return ds

# fig, axes = plt.subplots(5, 15, figsize=(30, 10))
# for row, ax_cols in enumerate(axes):
#     for ax in ax_cols:
#         ax.imshow(aug_model(X_train_full[row]))
#         ax.axis('off')
# plt.show()

## 3.1 Fix pretrained model

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_sample_weight

X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, stratify=y_train_full)
sample_weight_train = compute_sample_weight(class_weight='balanced', y=y_train.argmax(1))
sample_weight_val   = compute_sample_weight(class_weight='balanced', y=y_val.argmax(1))

train_ds = preprocess(tf.data.Dataset.from_tensor_slices((X_train, y_train, sample_weight_train)), True, BATCH_SIZE)
val_ds   = preprocess(tf.data.Dataset.from_tensor_slices((X_val, y_val, sample_weight_val)), False, BATCH_SIZE)
test_ds  = preprocess(tf.data.Dataset.from_tensor_slices(X_test), False, BATCH_SIZE)

print("- train.shape:", X_train.shape, y_train.shape)
print("- val.shape:", X_val.shape, y_val.shape)
print("- test.shape:", X_test.shape)

- train.shape: (3207, 512, 512, 3) (3207, 88)
- val.shape: (1070, 512, 512, 3) (1070, 88)
- test.shape: (2154, 512, 512, 3)


In [6]:
from tensorflow_addons.metrics import F1Score

def build_model(n_classes, strategy):
    with strategy.scope():
        base_model = keras.applications.EfficientNetB0(include_top=False, input_shape=input_shape)
        base_model.trainable = False

        inputs  = keras.Input(input_shape)
        hidden  = base_model(inputs, training=False)
        hidden  = keras.layers.GlobalAveragePooling2D()(hidden)
        outputs = keras.layers.Dense(n_classes, activation='softmax')(hidden)
        model   = keras.Model(inputs, outputs)
        
        model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=[F1Score(num_classes=n_classes, average='macro')])
    return model, base_model

In [8]:
from analysis_tools.modeling import *

model, base_model = build_model(n_classes, strategy)
epochs = 200
history = model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=get_callbacks(patience=50, plot_path=join(PATH.result, 'proposed3', 'fix_pretrained_model_nadam_lr_scheduler'), init_lr=2e-3, epochs=epochs, warmup_epoch=30, min_lr=1e-3))

Epoch 1/200
learning rate:  6.666666666666667e-05
INFO:tensorflow:batch_all_reduce: 2 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 2 all-reduces with algorithm = nccl, num_packs = 1
101/101 [==============================] - 43s 190ms/step - loss: 4.3492 - f1_score: 0.0400 - val_loss: 4.1480 - val_f1_score: 0.0767
Epoch 2/200
learning rate:  0.00013333333333333334
101/101 [==============================] - 13s 98ms/step - loss: 3.8217 - f1_score: 0.1506 - val_loss: 3.4854 - val_f1_score: 0.1563
Epoch 3/200
learning rate:  0.0002
101/101 [==============================] - 13s 102ms/step - loss: 3.1721 - f1_score: 0.1703 - val_loss: 2.8498 - val_f1_score: 0.1752
Epoch 4/200
learning rate:  0.00026666666666666668
101/101 [==============================] - 13s 99ms/step - loss: 2.6185 - f1_score: 0.1845 - val_loss: 2.3609 - val_f1_score: 0.2169
Epoch 5/200
learning rate:  0.00033333333333333332
101/101 [==============================] - 14s 99ms/step -

101/101 [==============================] - 13s 98ms/step - loss: 0.5182 - f1_score: 0.7102 - val_loss: 1.0383 - val_f1_score: 0.5366
Epoch 44/200
learning rate:  0.0019176472
101/101 [==============================] - 13s 100ms/step - loss: 0.4998 - f1_score: 0.7117 - val_loss: 1.0594 - val_f1_score: 0.5291
Epoch 45/200
learning rate:  0.00191176473
101/101 [==============================] - 12s 100ms/step - loss: 0.5042 - f1_score: 0.7230 - val_loss: 1.1501 - val_f1_score: 0.5019
Epoch 46/200
learning rate:  0.00190588238
101/101 [==============================] - 13s 99ms/step - loss: 0.4705 - f1_score: 0.7378 - val_loss: 1.1389 - val_f1_score: 0.5546
Epoch 47/200
learning rate:  0.0019
101/101 [==============================] - 12s 99ms/step - loss: 0.4988 - f1_score: 0.7063 - val_loss: 1.0944 - val_f1_score: 0.5029
Epoch 48/200
learning rate:  0.0018941178
101/101 [==============================] - 13s 101ms/step - loss: 0.4705 - f1_score: 0.7164 - val_loss: 1.1095 - val_f1_score: 

Epoch 90/200
learning rate:  0.00164705887
101/101 [==============================] - 12s 98ms/step - loss: 0.2641 - f1_score: 0.8130 - val_loss: 1.2227 - val_f1_score: 0.5567
Epoch 91/200
learning rate:  0.00164117652
101/101 [==============================] - 12s 100ms/step - loss: 0.2736 - f1_score: 0.8193 - val_loss: 1.1782 - val_f1_score: 0.5496
Epoch 92/200
learning rate:  0.00163529417
101/101 [==============================] - 12s 100ms/step - loss: 0.2585 - f1_score: 0.8258 - val_loss: 1.2748 - val_f1_score: 0.5251
Epoch 93/200
learning rate:  0.00162941182
101/101 [==============================] - 12s 99ms/step - loss: 0.2637 - f1_score: 0.8233 - val_loss: 1.3168 - val_f1_score: 0.5368
Epoch 94/200
learning rate:  0.00162352948
101/101 [==============================] - 13s 100ms/step - loss: 0.2680 - f1_score: 0.8245 - val_loss: 1.1637 - val_f1_score: 0.5835
Epoch 95/200
learning rate:  0.00161764713
101/101 [==============================] - 14s 109ms/step - loss: 0.2785 -

## 3.2 Fine-tuning

In [10]:
with strategy.scope():
    base_model.trainable = True
    model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=[F1Score(num_classes=n_classes, average='macro')])
epochs = 200
history = model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=get_callbacks(patience=50, plot_path=join(PATH.result, 'proposed3', 'fix_pretrained_model_nadam_lr_scheduler'), init_lr=2e-4, epochs=epochs, warmup_epoch=30, min_lr=1e-4))

Epoch 1/200
learning rate:  6.6666666666666666e-06
INFO:tensorflow:batch_all_reduce: 213 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 213 all-reduces with algorithm = nccl, num_packs = 1
101/101 [==============================] - 114s 499ms/step - loss: 0.1909 - f1_score: 0.8554 - val_loss: 1.2678 - val_f1_score: 0.5980
Epoch 2/200
learning rate:  1.3333333333333333e-05
101/101 [==============================] - 39s 362ms/step - loss: 0.2275 - f1_score: 0.8356 - val_loss: 1.2652 - val_f1_score: 0.5674
Epoch 3/200
learning rate:  2e-05
101/101 [==============================] - 39s 363ms/step - loss: 0.2527 - f1_score: 0.8173 - val_loss: 1.2961 - val_f1_score: 0.5992
Epoch 4/200
learning rate:  2.6666666666666667e-05
101/101 [==============================] - 38s 363ms/step - loss: 0.3601 - f1_score: 0.7790 - val_loss: 1.3069 - val_f1_score: 0.5455
Epoch 5/200
learning rate:  3.3333333333333335e-05
101/101 [==============================] - 39s 366m

101/101 [==============================] - 39s 364ms/step - loss: 0.0840 - f1_score: 0.9313 - val_loss: 1.0275 - val_f1_score: 0.7580
Epoch 44/200
learning rate:  0.000191764702
101/101 [==============================] - 39s 364ms/step - loss: 0.0513 - f1_score: 0.9521 - val_loss: 1.0205 - val_f1_score: 0.7543
Epoch 45/200
learning rate:  0.000191176456
101/101 [==============================] - 40s 363ms/step - loss: 0.0632 - f1_score: 0.9611 - val_loss: 0.9355 - val_f1_score: 0.7672
Epoch 46/200
learning rate:  0.000190588224
101/101 [==============================] - 40s 363ms/step - loss: 0.0395 - f1_score: 0.9746 - val_loss: 0.8553 - val_f1_score: 0.7695
Epoch 47/200
learning rate:  0.000189999992
101/101 [==============================] - 39s 362ms/step - loss: 0.0226 - f1_score: 0.9819 - val_loss: 0.9221 - val_f1_score: 0.7564
Epoch 48/200
learning rate:  0.000189411759
101/101 [==============================] - 40s 364ms/step - loss: 0.0215 - f1_score: 0.9775 - val_loss: 0.8391

101/101 [==============================] - 39s 372ms/step - loss: 0.0014 - f1_score: 1.0000 - val_loss: 0.8003 - val_f1_score: 0.8376
Epoch 90/200
learning rate:  0.000164705882
101/101 [==============================] - 40s 362ms/step - loss: 0.0117 - f1_score: 0.9911 - val_loss: 0.8628 - val_f1_score: 0.8130
Epoch 91/200
learning rate:  0.000164117635
101/101 [==============================] - 40s 375ms/step - loss: 0.0045 - f1_score: 0.9892 - val_loss: 0.9166 - val_f1_score: 0.8345
Epoch 92/200
learning rate:  0.000163529403
101/101 [==============================] - 38s 362ms/step - loss: 0.0275 - f1_score: 0.9832 - val_loss: 0.9996 - val_f1_score: 0.7827
Epoch 93/200
learning rate:  0.000162941171
101/101 [==============================] - 38s 360ms/step - loss: 0.0979 - f1_score: 0.9537 - val_loss: 0.9701 - val_f1_score: 0.7818
Epoch 94/200
learning rate:  0.000162352939
101/101 [==============================] - 38s 362ms/step - loss: 0.0525 - f1_score: 0.9545 - val_loss: 1.4280

Epoch 135/200
learning rate:  0.000138235293
101/101 [==============================] - 38s 360ms/step - loss: 0.0318 - f1_score: 0.9781 - val_loss: 0.7973 - val_f1_score: 0.8078
Epoch 136/200
learning rate:  0.000137647061
101/101 [==============================] - 40s 366ms/step - loss: 0.0129 - f1_score: 0.9848 - val_loss: 0.8051 - val_f1_score: 0.8168
Epoch 137/200
learning rate:  0.000137058814
101/101 [==============================] - 39s 363ms/step - loss: 0.0126 - f1_score: 0.9860 - val_loss: 0.7639 - val_f1_score: 0.8304
Epoch 138/200
learning rate:  0.000136470582
101/101 [==============================] - 39s 362ms/step - loss: 0.0134 - f1_score: 0.9911 - val_loss: 0.8884 - val_f1_score: 0.7938
Epoch 139/200
learning rate:  0.00013588235
101/101 [==============================] - 38s 363ms/step - loss: 0.0446 - f1_score: 0.9712 - val_loss: 0.9143 - val_f1_score: 0.8076
Epoch 140/200
learning rate:  0.000135294104
101/101 [==============================] - 39s 366ms/step - l

# 4. Evaluation

In [11]:
submission_file_path = join(PATH.output, 'proposed3_fine_tuning_nadam_lr_scheduler.csv')

pred_test = model.predict(test_ds)
submission = pd.read_csv(join(PATH.input, 'sample_submission.csv'), index_col=0)
submission['label'] = y_enc.inverse_transform(pred_test)
submission.to_csv(submission_file_path)
submission

,label
index,
0,tile-glue_strip
1,grid-good
2,transistor-good
3,tile-gray_stroke
4,tile-good
...,...
2149,tile-gray_stroke
2150,screw-good
2151,grid-good


# 5. Submission

In [12]:
from dacon_submit_api.dacon_submit_api import post_submission_file

result = post_submission_file(
    submission_file_path,
    '137ff236e305f302819b930b3b5b72e948603f23c5249a516c32b536d5187a03', 
    '235894', 
    '어스름한 금요일 밤에', 
    'proposed3_fine_tuning_nadam_lr_scheduler'
)

{'isSubmitted': True, 'detail': 'Success'}
